In [ ]:
#https://motius.breezy.hr/p/0b622aa1163e01-join-our-tech-community-as-working-student-m-f-d/apply?utm_source=pt-linkedin&utm_medium=job-posting

In [ ]:
"""import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import os
os.environ['CURL_CA_BUNDLE'] = ''"""

In [ ]:
from transformers import  AutoModel,AutoTokenizer,AutoModelForSequenceClassification ,AutoConfig, Trainer,TrainingArguments,\
BitsAndBytesConfig,pipeline,default_data_collator,DataCollatorWithPadding,DataCollatorForLanguageModeling
from transformers.utils import move_cache
from chat_template_utils import get_json_schema
from llama_cpp import Llama
from utils import *
from peft import *
import datasets
from datasets import Features, Value
import torchmetrics
import torch
from torch.utils.data.dataloader import DataLoader,Dataset
from hqq.engine.hf import HQQModelForCausalLM
from hqq.models.hf.base import AutoHQQHFModel
from huggingface_hub import snapshot_download
import deepspeed
import os
import pandas as pd
import json
from var_dump import var_dump
from bs4 import BeautifulSoup,PageElement,Comment
import matplotlib.pyplot as plt
import seaborn as sns
import langchain
from bertopic import BERTopic

cache_dir='/var/tmp/.cache/' #'/proj/ciptmp/ix05ogym/.cache/'
output_dir = cache_dir+'outputs/'

move_cache(cache_dir)

def pd_full_screen():
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    
def pd_normal_screen():
    pd.set_option('display.max_rows', 10)
    pd.set_option('display.max_columns', 5)



In [ ]:
df['query'] = df['query'].apply(lambda x: str(x))
df['element'] = df['element'].apply(lambda x: str(x))

df['query'] = df['query'].apply(lambda x: x.replace('\xa0', ' '))
df['element'] = df['element'].apply(lambda x: x.replace('\xa0', ' '))

In [ ]:
from sentence_transformers  import SentenceTransformer
#from transformers import AutoModel
model_name= 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
#sentence-transformers/all-MiniLM-L12-v2
embdeding_tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=cache_dir)

#embeding_model= AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2',cache_dir=cache_dir)#.to('cuda')
embeding_model= SentenceTransformer(model_name,cache_folder=cache_dir)#.to('cuda')
#guide_model= SentenceTransformer(model_name,cache_folder=cache_dir)#.to('cuda')



embeding_model.eval()
embeding_model

In [ ]:
def print_df(df):
    for i,x in df.iterrows():
        print(i)
        for k,v in x.items():
            if k=='query':
                continue
            print(k,v)
            
        print('---------------------')
    
c = df['query'].str.lower().str.contains('search')==True
df.loc[c,'label']='search form'
c = df['query'].str.lower().str.contains('submit application')==True
df.loc[c,'label']='application form'
c = df['query'].str.lower().str.contains('login')==True
df.loc[c,'label']='login form'
c = df['query'].str.lower().str.contains('register')==True
df.loc[c,'label']='register form'

labeled = df.dropna().reset_index(drop=True)
print(labeled['label'].value_counts())
print_df(labeled)

In [ ]:
labeled.loc[24,'label']='application form'
labeled.loc[34,'label']='application form'
labeled.loc[45,'label']='application form'
labeled.loc[100,'label']='application form'
labeled.loc[90,'label']='application form'
labeled.loc[53,'label']='application form'


labeled.loc[87,'label']='next'
labeled.loc[86,'label']='register form'
labeled.loc[80,'label']='***'

labeled.loc[14,'label']='login form'
labeled.loc[32,'label']='login form'
labeled.loc[40,'label']='login form'


labeled.loc[32,'label']='login form'
labeled.loc[27,'label']='newsletter'



c = labeled['label']=='search form'
md = labeled.loc[[24,45,100,90,53,87,86,14,32,27,40,32,10,11,12,13,17,23,28],:].copy().reset_index(drop=True)


print_df(md)

In [ ]:
md.to_excel('true_form.xlsx')

In [ ]:
labels = labeled['label'].unique()
index2label = dict(enumerate(labels))
label2index= dict(zip(index2label.values(),index2label.keys()))

label2index

In [ ]:
labeled['label'] = labeled['label'].map(lambda x:label2index[x])


In [ ]:
from sentence_transformers.trainer import SentenceTransformerTrainer,SentenceTransformerTrainingArguments
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


In [ ]:
"""#data.loc[:,'label']=1
features = Features({
    'query': Value('string'),
    'element': Value('string'),
    'answer': Value('string'),
    'label': Value('int64')
})
train_dataset = datasets.Dataset.from_pandas(data,features)

train_dataset = train_dataset.remove_columns(['answer'])#,'label'])
"""

"""
train_dataset = train_dataset.map(lambda x:
    {
        'element':embdeding_tokenizer(x['element'],return_tensors='pt',truncation=True,max_length=512),
        'label':x['label']
        
    }
    ,
    remove_columns=['query']

    
    )
    
"""

In [ ]:
#labeled.loc[4,'query']="ss"
#labeled.loc[4,'label']=0
features = Features({
    'query': Value('string'),
    'element': Value('string'),
    'label': Value('int64')
})
"""labeled.loc[0:10,'query']="sss"
labeled.loc[8,'query']="skmklmss"""
"""
labeled.loc[0:10,'label']=0
labeled.loc[2,'label']=1
"""

train_dataset = datasets.Dataset.from_pandas(labeled[:128],features)
train_dataset = train_dataset.remove_columns(['element'])
train_dataset

In [ ]:
train_dataset[0]

In [ ]:
"""def pd_full_screen():
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    
def pd_normal_screen():
    pd.set_option('display.max_rows', 10)
    pd.set_option('display.max_columns', 5)
pd_full_screen()
"""
train_dataset[0]

In [ ]:
from sentence_transformers.losses import BatchSemiHardTripletLoss as myloss
trainer  =  SentenceTransformerTrainer(model=embeding_model,
                           train_dataset=train_dataset,
                           tokenizer=embdeding_tokenizer,
                           loss= myloss(embeding_model)#,384,len(u)),#,'sentence-transformers/all-MiniLM-L12-v2',True),
                           ,args=SentenceTransformerTrainingArguments(
                               output_dir=output_dir,
                               num_train_epochs=20,
                               per_device_train_batch_size=32,
                               logging_steps=1,
                               #learning_rate=1e-9
                               
                           )
                           
                           )


trainer.train()

In [ ]:
#c = test_dataset['query'].isna()
#test_dataset.loc[c,'query']='cv'
#test_dataset.to_excel('forms.xlsx',index=0)

In [ ]:
test_dataset = pd.read_excel('forms.xlsx')
c = test_dataset['label'].notna()
test_dataset = test_dataset.loc[c,:].reset_index(drop=True)

tp = test_dataset #train_dataset.to_pandas()
def compute_metric():
    embeding_model.eval()
    acc=0
    
    with torch.no_grad():
        
        l=embeding_model.encode(LABEL_INDEX_TO_KEY)
        l = torch.tensor(l)
        print(l.shape)
        
        for i,x in tp.iterrows():
            #print(x)
            #for i,x in train_dataset.iterrows():
            #te = embdeding_tokenizer([x['element']],max_length=512,truncation=True,return_tensors='pt')['input_ids']
            #print(i)
            #x['label']
            o = embeding_model.encode( [x['query']] )
            o2 = embeding_model.encode( x['element'] )
            
            o = torch.tensor(o)
            o2 = torch.tensor(o2)
            
            sim = torch.cosine_similarity(l,o)
            #print(o2.shape)
            #sim = torch.cosine_similarity(o2,o,0)

            m = sim.argmax()
            tp.loc[i,'sudo_label']=LABEL_INDEX_TO_KEY[m]
            #tp.loc[i,'sudo_label_value']=sim
            if LABEL_INDEX_TO_KEY[m]==x['label']:
                acc+=1
                pass
            #print(LABEL_INDEX_TO_KEY[sim.argmax()],x['label'])
            
           
             
    
    acc/=len(test_dataset)
    print("acc:",acc)
    
    
    
compute_metric()

In [ ]:
test_dataset.loc[11,'query']='cv *'
tp

In [ ]:
for i,x in tp.iterrows():
            print(x['query'])
            print(x['element'])
            print(x['sudo_label'])
            #print(x['sudo_label_value'])
            print('-----------------------------------')